# Route Planner: A Prolog Application with LangChain Integration

## Overview
The `routes.pl` application is a travel route planning system implemented in Prolog that demonstrates logical reasoning capabilities integrated with LangChain. It serves as an example of how Prolog's declarative programming paradigm can enhance LLM applications with powerful reasoning.

## Key Features
- **Transport Network Modeling:** Intuitively defines connections between cities with various transport types (train, plane, ferry).  
- **Route Finding:** Discovers all possible routes between origin and destination cities.  
- **Optimization:** Calculates fastest and cheapest routes based on time and cost parameters.  
- **Constraint Handling:** Applies constraints like maximum time, maximum cost, and number of connections.  
- **Transport Preferences:** Filters routes based on preferred transport types.  
- **Comprehensive Query Interface:** Provides predicates for different query types and formats.  

## Technical Highlights

- Uses Prolog's backtracking to explore all possible routes.  
- Implements cycle detection to prevent infinite loops.  
- Provides helper predicates for visualizing and analyzing routes.  
- Structures data for easy integration with Python.  
- Returns results in formats compatible with LangChain interfaces.  

## Integration with LangChain
When integrated with LangChain or LangGraph, this application enables:

- Natural language queries about travel routes.  
- Reasoning about optimal travel plans based on constraints.  
- Explanations of why certain routes are preferred.  
- Step-by-step travel planning with LLM guidance.  

## Setup

**Prerequisites**

- Python 3.10 or later
- SWI-Prolog installed on your system
- The following python libraries installed:
    - langchain 0.3.0 or later
    - janus-swi 1.5.0 or later
    - pydantic 0.2.0 or later

The Prolog interfase with LangChain can be installed using pip:

In [1]:
#!pip install langchain-prolog

from langchain_prolog import PrologConfig, PrologRunnable, PrologTool

## Instantiation
The most important classes in langchain-prolog are: `PrologConfig`, `PrologRunnable` and `PrologTool` 
- `PrologConfig` sets the configuration for the Prolog interpreter. The only mandatory field is the path to the Prolog script to be used.
- `PrologRunnable.create_schema` defines a Pydantic schema to be used to pass arguments to the Prolog predicates. It is optional, but recomended.
- `PrologTool` wraps the Prolog script with the interfase to LangChain/LaqngGraph. I supports all the methods of the `Tool` class and tracing capabilities of LangSmith.

In [2]:
schema = PrologRunnable.create_schema('query_route', ['query_type', 'from', 'to', 'options', 'results'])

config = PrologConfig(
            rules_file='routes.pl',
            query_schema=schema,
            default_predicate="query_route"
        )

planner_tool = PrologTool(
    prolog_config=config,
    name="travel_planner",
    description="""
        Query travel routes using Prolog.
        Input can be a query string like 'query_route(all, paris, london,[], Results)'.
        You have to specify 3 parameters:
            - query_type: can be 'all', 'fastest' or 'cheapest'
            - from: the city where the travel starts. Must be all lower case.
            - to: the city where the travel ends. Must be all lower case.
            - Results: This will be the key for the results
        The query will return:
            - 'False' if there are no routes availables
            - A dictionary with 'Results' as the key and a list ouf possible routes as the value
        Do not use quotes.
    """,
)

## Invocation
If a schema is defined, we can pass a dictionary using the names of the parameters in the schema as the keys in the dictionary. The values can represent Prolog variables (uppercase first letter) or strings (lower case first letter). A `None` value is interpreted as a variable and replaced with the key capitalized:

In [3]:
planner_tool.invoke(
    {
        'query_type': 'all', 
        'from': 'paris',
        'to': 'lisbon',
        'options': [],
        'results': None,
    }
)

[{'Results': [{'time': 11,
    'transport': [{'time': 2, 'type': 'plane', 'cost': 180},
     {'time': 2, 'type': 'plane', 'cost': 140},
     {'time': 3, 'type': 'train', 'cost': 80},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'rome', 'barcelona', 'madrid', 'lisbon'],
    'cost': 490},
   {'time': 8.5,
    'transport': [{'time': 1.5, 'type': 'plane', 'cost': 120},
     {'time': 3, 'type': 'train', 'cost': 80},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'barcelona', 'madrid', 'lisbon'],
    'cost': 290},
   {'time': 6,
    'transport': [{'time': 2, 'type': 'train', 'cost': 150},
     {'time': 4, 'type': 'train', 'cost': 90}],
    'route': ['paris', 'madrid', 'lisbon'],
    'cost': 240}]}]

### Using an LLM and function calling

Yor can use any LangChain chat model that supports tool calling.

In [4]:
#!pip install python-dotenv

from dotenv import find_dotenv
from dotenv import load_dotenv

load_dotenv(find_dotenv(), override=True)

#!pip install langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

To use the Prolog tool, bind it to the LLM model:

In [5]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools([planner_tool])

and then query the model:

In [6]:
query = "I want to tavel from Paris to Lisbon. What are my options?"

tries = 1
while tries <= 10:
    try:
        
        # The user query is pass in a list of messages
        messages = [HumanMessage(query)]
        response = llm_with_tools.invoke(messages)
        
         # The LLM will respond with a tool call request if needed
        if hasattr(response, 'tool_calls'):
            messages.append(response)
            
            # The tool takes this request and queries the Prolog database:
            tool_msg = planner_tool.invoke(response.tool_calls[0])
            messages.append(tool_msg)
            
            #The tool returns a list with all the solutions for the query
            response = llm_with_tools.invoke(messages)
            
        print(response.content)
        break
        
    except:
        tries += 1
        if tries > 10:
            print("Could not get an answer")

Here are your travel options from Paris to Lisbon:

1. **Route**: Paris → Rome → Barcelona → Madrid → Lisbon
   - **Total Time**: 11 hours
   - **Transport Options**:
     - Plane: 2 hours, Cost: €180
     - Plane: 2 hours, Cost: €140
     - Train: 3 hours, Cost: €80
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €490

2. **Route**: Paris → Barcelona → Madrid → Lisbon
   - **Total Time**: 8.5 hours
   - **Transport Options**:
     - Plane: 1.5 hours, Cost: €120
     - Train: 3 hours, Cost: €80
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €290

3. **Route**: Paris → Madrid → Lisbon
   - **Total Time**: 6 hours
   - **Transport Options**:
     - Train: 2 hours, Cost: €150
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €240

Choose the option that best fits your schedule and budget!


## Chaining

### Using an Agent
To use a Prolog tool with an agent, pass it to the agent's constructor:

In [7]:
#!pip install langgraph 

from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [planner_tool])

The agent takes the query and use the Prolog tool if needed. Then the agent receives​ the tool response and generates the answer:

In [8]:
verbose = False
tries = 1
while tries <= 10:
    try:
        
        # The user query is pass as a dictionary with the list of messages
        inputs = {"messages": [("human", query)]}
    
        if verbose:
            for step in agent_executor.stream(inputs, stream_mode="values"):
                message = step["messages"][-1]
                message.pretty_print()
        
        else:
            outputs = agent_executor.invoke(inputs)
            results = outputs["messages"][-1]
            results.pretty_print()
            
        break
        
    except:
        tries += 1
        if tries > 10:
            print("Could not get an answer")

================================== Ai Message ==================================

Here are your travel options from Paris to Lisbon:

1. **Route**: Paris → Rome → Barcelona → Madrid → Lisbon
   - **Total Time**: 11 hours
   - **Transport Options**:
     - Plane: 2 hours, Cost: €180
     - Plane: 2 hours, Cost: €140
     - Train: 3 hours, Cost: €80
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €490

2. **Route**: Paris → Barcelona → Madrid → Lisbon
   - **Total Time**: 8.5 hours
   - **Transport Options**:
     - Plane: 1.5 hours, Cost: €120
     - Train: 3 hours, Cost: €80
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €290

3. **Route**: Paris → Madrid → Lisbon
   - **Total Time**: 6 hours
   - **Transport Options**:
     - Train: 2 hours, Cost: €150
     - Train: 4 hours, Cost: €90
   - **Total Cost**: €240

You can choose any of these options based on your preference for time and cost!


## API reference

See https://langchain-prolog.readthedocs.io/en/latest/modules.html for detail.